In [46]:
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from networkx.algorithms import community

# Slow Zone Modularity Analysis

In [2]:
bronx_all = pd.read_csv('all_bronx_edges.csv')
queens_all = pd.read_csv('all_queens_edges.csv')

In [3]:
bronx_all.head()

,Unnamed: 0,SegmentID,NodeIDFrom,NodeIDTo,LCB2010,RCB2010,persons_injured,pedestrians_injured,volume_count
0,0,78126,47740,9045677,3001,2000,2.0,0.0,0.0
1,1,79796,48679,48678,2000,1004,0.0,0.0,0.0
2,2,77356,47288,47822,2000,3006,0.0,0.0,0.0
3,3,73490,45034,45038,1004,1008,0.0,0.0,0.0
4,4,174633,84655,45050,1003,1001,0.0,0.0,0.0


In [5]:
bronx_all_graph = nx.from_pandas_edgelist(bronx_all,source='NodeIDFrom',target='NodeIDTo',edge_attr=['SegmentID','persons_injured','pedestrians_injured','volume_count'])

In [10]:
bronx_all_graph[47740][9045677]

{'SegmentID': 78126,
 'pedestrians_injured': 0.0,
 'persons_injured': 2.0,
 'volume_count': 0.0}

In [38]:
node_indicators = pd.read_csv('node_slow_zone_indicators.csv').fillna(0)
node_indicators.sort_values('zone_ind',ascending=False).head()

,Unnamed: 0,NODEID,name,year,zone_ind
114758,114759,9015180,Riverdale SZ,2013.0,1
103570,103571,9000931,Alphabet City Tompkins Square Park SZ,2014.0,1
103562,103563,9000923,Alphabet City Tompkins Square Park SZ,2014.0,1
103558,103559,9000919,Alphabet City Tompkins Square Park SZ,2014.0,1
103554,103555,9000915,Alphabet City Tompkins Square Park SZ,2014.0,1


In [64]:
node_dict = pd.Series(node_indicators.zone_ind,index=node_indicators.NODEID).to_dict()
node_dict
#need to get rid of NaNs?
nx.set_node_attributes(bronx_all_graph,node_dict,'slow_zone_ind')

https://stackoverflow.com/questions/29897243/graph-modularity-in-python-networkx

In [68]:
A_unweighted = nx.to_numpy_matrix(bronx_all_graph)
node_degrees = bronx_all_graph.degree(weight='persons_injured')
node_indicators = nx.get_node_attributes(bronx_all_graph,'slow_zone_ind')
n_edges = A_unweighted.sum()/2

In [84]:
bronx_all_modmtx = nx.modularity_matrix(bronx_all_graph, nodelist=None, weight='persons_injured')

In [85]:
print(bronx_all_modmtx)

[[-1.45586897e-03  1.99927207e+00 -7.27934486e-04 ... -3.63967243e-04
   0.00000000e+00  0.00000000e+00]
 [ 1.99927207e+00 -3.63967243e-04 -3.63967243e-04 ... -1.81983621e-04
   0.00000000e+00  0.00000000e+00]
 [-7.27934486e-04 -3.63967243e-04 -3.63967243e-04 ... -1.81983621e-04
   0.00000000e+00  0.00000000e+00]
 ...
 [-3.63967243e-04 -1.81983621e-04 -1.81983621e-04 ... -9.09918107e-05
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]]


# Betweenness Centralities

In [73]:
bronx_counts = pd.read_csv('bronx_traffic_edges.csv')
queens_counts = pd.read_csv('queens_traffic_edges.csv')

In [74]:
bronx_counts.head()

,Unnamed: 0,SegmentID,NodeIDFrom,NodeIDTo,LCB2010,RCB2010,persons_injured,pedestrians_injured,volume_count
0,188,81384,49666,54110,1025,1026,0.0,0.0,9021.0
1,346,88039,53649,53715,4006,1004,0.0,0.0,3027.0
2,422,69588,42728,42732,1018,1007,0.0,0.0,4951.0
3,494,88020,53640,53641,2000,1001,0.0,0.0,8288.0
4,502,88031,53645,53653,1003,3000,4.0,1.0,4763.0


In [75]:
bronx_nonzero_graph = nx.from_pandas_edgelist(bronx_counts,source='NodeIDFrom',target='NodeIDTo',edge_attr=['SegmentID','persons_injured','pedestrians_injured','volume_count'])

In [76]:
crash_injury_degrees = bronx_nonzero_graph.degree(weight='persons_injured')
volume_degrees = bronx_nonzero_graph.degree(weight='volume_count')

In [82]:
flow_centrality = nx.betweenness_centrality(bronx_nonzero_graph,weight='volume_count')
crash_centrality = nx.betweenness_centrality(bronx_nonzero_graph,weight='persons_injured')